# Password Security Suite: Strength Checker, Encryption, and Decryption

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


# Password Strength Check

This project evaluates the strength of a password based on various criteria and provides feedback. Below is a simple explanation of how the code works:

### Key Points

### **1. Using Gradio for Web Interface**
   - **Gradio** is used to create a user-friendly web interface for the password checker.
   - The interface includes input fields for the password and a checkbox to toggle the visibility of the entered password.

### **2. Password Input Handling**
   - A **textbox** is provided for users to input their password (`password_input`).
   - A **checkbox** (`show_password`) toggles between showing or hiding the password characters.
   - When the password changes, or the visibility toggles, the program evaluates the password strength in real-time.
   - All the entered passwords are saved in password.txt file

### **3. Evaluating Password Strength**
   - The function `check_password_strength` uses **regular expressions** to assess specific password criteria:
     - **Length:** Checks if the password is at least 8 characters long and gives additional points for 12+ characters.
     - **Uppercase Letters:** Verifies the presence of uppercase letters (e.g., `A-Z`).
     - **Lowercase Letters:** Verifies the presence of lowercase letters (e.g., `a-z`).
     - **Numbers:** Checks if the password contains digits (`0-9`).
     - **Special Characters:** Looks for special characters (e.g., `!@#$%^&*`).

### **4. Calculating Password Strength**
   - The password strength is scored out of 6 based on the above criteria.
   - A **percentage** is calculated for the score, representing the strength:
     - **Weak (0-33%)**: Red meter.
     - **Moderate (34-66%)**: Orange meter.
     - **Strong (67-100%)**: Green meter.



### Benefits
- Helps users create strong and secure passwords.
- Provides detailed feedback for improvement.
- Interactive and easy to use.


In [ ]:
import gradio as gr
import re

def check_password_strength(password, show_password):
    """
    Evaluate password strength based on various criteria
    Returns password display value, strength score, meter color, and detailed feedback
    """
    # Handle password visibility
    display_password = password if show_password else "*" * len(password)

    strength = 0
    feedback = []

    # Check length
    if len(password) < 8:
        feedback.append("❌ Password should be at least 8 characters long")
    else:
        strength += 1
        feedback.append("✅ Good length")
        if len(password) >= 12:
            strength += 1
            feedback.append("✅ Excellent length (12+ characters)")

    # Check for uppercase letters
    if re.search(r'[A-Z]', password):
        strength += 1
        feedback.append("✅ Contains uppercase letters")
    else:
        feedback.append("❌ Should contain uppercase letters")

    # Check for lowercase letters
    if re.search(r'[a-z]', password):
        strength += 1
        feedback.append("✅ Contains lowercase letters")
    else:
        feedback.append("❌ Should contain lowercase letters")

    # Check for numbers
    if re.search(r'\d', password):
        strength += 1
        feedback.append("✅ Contains numbers")
    else:
        feedback.append("❌ Should contain numbers")

    # Check for special characters
    if re.search(r'[!@#$%^&*(),.?":{}|<>]', password):
        strength += 1
        feedback.append("✅ Contains special characters")
    else:
        feedback.append("❌ Should contain special characters")

    # Calculate strength percentage
    strength_percentage = (strength / 6) * 100

    # Determine strength label and color
    if strength_percentage <= 33:
        strength_label = "Weak"
        color = "#ff4d4d"  # Red
    elif strength_percentage <= 66:
        strength_label = "Moderate"
        color = "#ffa64d"  # Orange
    else:
        strength_label = "Strong"
        color = "#4CAF50"  # Green

    # Format feedback
    feedback_text = "\n".join(feedback)
    result = f"Strength: {strength_label} ({strength_percentage:.1f}%)\n\nDetailed Analysis:\n{feedback_text}"

    # Create HTML for the strength meter
    meter_html = f"""
    <div style="margin: 10px 0;">
        <div style="background-color: #ddd; height: 20px; border-radius: 10px; overflow: hidden;">
            <div style="width: {strength_percentage}%; height: 100%; background-color: {color};
                 transition: width 0.5s ease-in-out;">
            </div>
        </div>
    </div>
    """

    return display_password, meter_html, result

# Create Gradio interface with HTML
css = """
.gradio-container {max-width: 800px; margin: auto;}
.password-header {font-size: 24px; margin-bottom: 20px; text-align: center;}
"""

with gr.Blocks(css=css) as iface:
    gr.HTML('<h1 class="password-header">Password Strength Checker</h1>')

    with gr.Row():
        password_input = gr.Textbox(
            show_label=True,
            placeholder="Enter password to check",
            label="Password",
            type="password"
        )
        show_password = gr.Checkbox(
            label="Show Password",
            show_label=True,
            value=False
        )

    password_display = gr.Textbox(
        show_label=True,
        label="Password Display",
        interactive=False
    )

    strength_meter = gr.HTML()
    analysis_output = gr.Textbox(
        show_label=True,
        label="Password Analysis",
        interactive=False,
        lines=8
    )

    # Update all outputs in real-time
    password_input.change(
        check_password_strength,
        inputs=[password_input, show_password],
        outputs=[password_display, strength_meter, analysis_output],
    )

    # Update password display when show/hide is toggled
    show_password.change(
        check_password_strength,
        inputs=[password_input, show_password],
        outputs=[password_display, strength_meter, analysis_output],
    )

    gr.Examples(
        examples=[
            ["Password123!"],
            ["weakpass"],
            ["SuperStr0ng@2024"]
        ],
        inputs=password_input,
    )

if __name__ == "__main__":
    iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6ff99cdba22f18912d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Password Encryption and decryption using Symmetric Encrytion Method



### 1. **Symmetric Encryption**
- **Definition**: A cryptographic technique where the same key is used for both encrypting and decrypting data.

- Uses the `Fernet` class from the `cryptography` library to implement symmetric encryption.

---

### 2. **Encryption**
- **Process**:
  1. Converts plaintext data (e.g., passwords) into ciphertext using an encryption algorithm and a secret key.
  2. Ciphertext is unreadable without the key.
- **Purpose**:
  - Protects sensitive data by ensuring it cannot be read without decryption.

---

### 3. **Decryption**
- **Process**:
  1. Converts ciphertext back into readable plaintext using the same secret key.
  2. This process reverses encryption.
- **Requirement**:
  - The original secret key is necessary to perform decryption.

---

### 4. **Role of the Secret Key**
- The secret key is **central** to the encryption and decryption process:
  1. **Encryption**: Encodes plaintext into ciphertext securely.
  2. **Decryption**: Decodes ciphertext back to plaintext.
- **Security Dependency**:
  - If the key is lost, encrypted data cannot be decrypted.
  - If the key is compromised, encrypted data becomes vulnerable.

---

### 5. **File Handling**
- Files are used to store:
  - **Input Data**: Plaintext passwords (`password.txt`).
  - **Encrypted Data**: Encrypted passwords (`encrypted_passwords.txt`).
  - **Decrypted Data**: Plaintext passwords retrieved after decryption (`decrypted_passwords.txt`).
  - **Encryption Key**: The secret key (`secret.key`) is stored separately for secure reuse.




## **Benefits**

1. **Security**:
   - Protects sensitive data (like passwords) by making it unreadable without the secret key.
   
2. **Compliance**:
   - Meets security best practices by avoiding plaintext password storage.

3. **Convenience**:
   - Automates encryption and decryption processes for efficient data handling.

4. **Data Integrity**:
   - Ensures that encrypted data cannot be tampered with without detection.

---

## **Limitations**

1. **Key Management**:
   - Losing the secret key results in permanent loss of encrypted data.
   - Hackers can decrypt the data if the secret key is known to the hackers

2. **Single Point of Failure**:
   - If the key is compromised, all encrypted data becomes vulnerable.

3. **Performance Overhead**:
   - Encrypting and decrypting large datasets can take time.

4. **Key Distribution**:
   - Securely sharing the key with authorized users is challenging.



In [1]:
from cryptography.fernet import Fernet

def generate_key(key_file='secret.key'):
    """
    Generates a symmetric encryption key and saves it to a file.

    :param key_file: Path to save the generated key.
    :return: The generated key.
    """
    key = Fernet.generate_key()
    with open(key_file, 'wb') as f:
        f.write(key)
    return key

def load_key(key_file='secret.key'):
    """
    Loads the symmetric encryption key from a file.

    :param key_file: Path to the key file.
    :return: The encryption key.
    """
    return open(key_file, 'rb').read()

def encrypt_passwords(input_file='password.txt', output_file='encrypted_passwords.txt', key_file='secret.key'):
    """
    Encrypts passwords from the input file and saves them to the output file.

    :param input_file: Path to the input file containing plaintext passwords.
    :param output_file: Path to the output file to save encrypted passwords.
    :param key_file: Path to save/load the encryption key.
    """
    try:
        key = load_key(key_file)
    except FileNotFoundError:
        key = generate_key(key_file)

    fernet = Fernet(key)

    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for line in infile:
                password = line.strip()
                if not password:
                    continue
                encrypted = fernet.encrypt(password.encode('utf-8')).decode('utf-8')
                outfile.write(f"Encrypted: {encrypted}\n")
        print(f"Encrypted passwords have been saved to '{output_file}'.")
    except FileNotFoundError:
        print(f"Input file '{input_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

def decrypt_passwords(encrypted_file='encrypted_passwords.txt', decrypted_file='decrypted_passwords.txt', key_file='secret.key'):
    """
    Decrypts passwords from the encrypted file and saves them to the decrypted file.

    :param encrypted_file: Path to the file containing encrypted passwords.
    :param decrypted_file: Path to save decrypted (plaintext) passwords.
    :param key_file: Path to load the encryption key.
    """
    try:
        key = load_key(key_file)
        fernet = Fernet(key)
    except FileNotFoundError:
        print(f"Key file '{key_file}' not found. Cannot decrypt passwords.")
        return

    try:
        with open(encrypted_file, 'r') as infile, open(decrypted_file, 'w') as outfile:
            for line in infile:
                if not line.startswith("Encrypted: "):
                    continue
                encrypted = line.strip().split("Encrypted: ")[1]
                decrypted = fernet.decrypt(encrypted.encode('utf-8')).decode('utf-8')
                outfile.write(f"{decrypted}\n")
        print(f"Decrypted passwords have been saved to '{decrypted_file}'.")
    except FileNotFoundError:
        print(f"Encrypted file '{encrypted_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# # Example usage:
# if __name__ == "__main__":
#     # Encrypt passwords
encrypt_passwords('password.txt', 'encrypted_passwords.txt', 'secret.key')

# Decrypt passwords
decrypt_passwords('encrypted_passwords.txt', 'decrypted_passwords.txt', 'secret.key')

Encrypted passwords have been saved to 'encrypted_passwords.txt'.
Decrypted passwords have been saved to 'decrypted_passwords.txt'.


#Password Encryption and Decryption using Hashing Algorithms

### 1. **Hashing**
- **Definition**:
  - A one-way cryptographic transformation of input data (e.g., a password) into a fixed-size string of characters, which represents the input uniquely.
  - Hashing is deterministic: the same input always produces the same hash.

---

### 2. **Hashing Algorithms**
- The code supports various hashing algorithms:
  - **MD5**:
    - Produces a 128-bit hash.
    - Fast but considered insecure due to vulnerabilities to collisions.
  - **SHA-1**:
    - Produces a 160-bit hash.
    - Deprecated for security-sensitive applications due to collision attacks.
  - **SHA-256**:
    - Produces a 256-bit hash.
    - Part of the SHA-2 family, offering strong security.
  - **SHA-384**:
    - Produces a 384-bit hash.
    - A stronger variant in the SHA-2 family.
  - **SHA-512**:
    - Produces a 512-bit hash.
    - Provides the highest level of security in the SHA-2 family.

---

### 3. **Hashing for Passwords**
- **Purpose**:
  - Hashing passwords ensures that even if the data is compromised, plaintext passwords cannot be retrieved.
- **Usage**:
  - Hashes are stored instead of plaintext passwords, and during authentication, the hash of the input password is compared with the stored hash.

---

## **Benefits of Hashing**

1. **Irreversibility**:
   - Hashing is a one-way operation. Original input cannot be derived from the hash.

2. **Fixed-Size Output**:
   - Regardless of input size, hashing algorithms produce a fixed-length hash.

3. **Efficiency**:
   - Hashing algorithms are computationally efficient for processing data.

4. **Integrity Checking**:
   - Useful for verifying data integrity (e.g., checksums).

5. **Scalability**:
   - Hashes can be generated for any size of input data.

---

## **Limitations of Hashing**

1. **Not Encryption**:
   - Hashing is irreversible, unlike encryption, which allows for decryption back to the original data.

2. **Vulnerable Algorithms**:
   - Older algorithms like MD5 and SHA-1 are vulnerable to collision attacks (different inputs producing the same hash).

3. **Brute Force Attacks**:
   - Hashes are susceptible to brute force or dictionary attacks if the input space is small.

4. **Lack of Salt**:
   - Hashes without added randomness (salt) are vulnerable to rainbow table attacks.

---

## **Hashing Algorithm Details**

### 1. **MD5**
- **Bit Length**: 128 bits
- **Speed**: Fast
- **Security**: Weak, vulnerable to collisions, not recommended for sensitive data.

### 2. **SHA-1**
- **Bit Length**: 160 bits
- **Speed**: Moderate
- **Security**: Deprecated due to collision vulnerabilities.

### 3. **SHA-256**
- **Bit Length**: 256 bits
- **Speed**: Slower than MD5 and SHA-1 but secure.
- **Security**: Strong, widely used.

### 4. **SHA-384**
- **Bit Length**: 384 bits
- **Speed**: Slower than SHA-256.
- **Security**: More secure than SHA-256.

### 5. **SHA-512**
- **Bit Length**: 512 bits
- **Speed**: Slowest but strongest among these algorithms.
- **Security**: Highest level of security for hashing.

---


In [ ]:
import hashlib

def hash_password(password, algorithm='sha256'):
    """
    Hashes a password using the specified hashing algorithm.

    :param password: The plaintext password to hash.
    :param algorithm: The hashing algorithm to use.
                      Supported algorithms: 'md5', 'sha1', 'sha256', 'sha384', 'sha512'
    :return: The hexadecimal hash of the password.
    """
    algorithms = {
        'md5': hashlib.md5,
        'sha1': hashlib.sha1,
        'sha256': hashlib.sha256,
        'sha384': hashlib.sha384,
        'sha512': hashlib.sha512
    }

    if algorithm.lower() not in algorithms:
        raise ValueError(f"Unsupported hashing algorithm: {algorithm}")

    hash_func = algorithms[algorithm.lower()]()
    hash_func.update(password.encode('utf-8'))
    return hash_func.hexdigest()

def hash_passwords(input_file='password.txt', output_file='hashed_passwords.txt'):
    """
    Reads passwords from the input file, hashes them using multiple algorithms,
    and writes the hashes to the output file.

    :param input_file: Path to the input file containing plaintext passwords.
    :param output_file: Path to the output file to save hashed passwords.
    """
    hashing_algorithms = ['md5', 'sha1', 'sha256', 'sha384', 'sha512']

    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for line in infile:
                password = line.strip()
                if not password:
                    continue  # Skip empty lines
                outfile.write(f"Password: {password}\n")
                for algo in hashing_algorithms:
                    hashed = hash_password(password, algo)
                    outfile.write(f"{algo.upper()}: {hashed}\n")
                outfile.write("\n")  # Add a newline for readability
        print(f"Hashed passwords have been saved to '{output_file}'.")
    except FileNotFoundError:
        print(f"Input file '{input_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
if __name__ == "__main__":
    hash_passwords('password.txt', 'hashed_passwords.txt')

Hashed passwords have been saved to 'hashed_passwords.txt'.


Encrypted passwords have been saved to 'encrypted_passwords.txt'.
Decrypted passwords have been saved to 'decrypted_passwords.txt'.
